<a href="https://colab.research.google.com/github/jeminjoachim/ai-newsletter-agent/blob/main/notebooks/ML/enterp_crypto_lit_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bertopic --quiet
!pip install llama-stack --quiet
!pip install llama-stack -U --quiet
!pip install -U bitsandbytes --quiet
!pip install bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.3/445.3 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 89.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado==6.5.1, but you have tornado 6.5.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from tqdm import tqdm
import numpy as np
import umap
import hdbscan
from umap import UMAP
from hdbscan import HDBSCAN

/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


In [ ]:
df = pd.read_csv("/content/entrep+crypto_lit.csv")
df.head()

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,Cited by,DOI,Link,Affiliations,Authors with affiliations,Abstract,Author Keywords,Publisher,Document Type,Source
0,"Mohamed, M.A.; Dirie, A.N.; Mohamud, A.B.; Far...","Mohamed, Mahad Abdiwali (60332580300); Dirie, ...",60332580300; 59238226700; 60331753000; 6033162...,A bibliometric study on cryptocurrency crowdfu...,2026,Discover Sustainability,7,1,67,NaN,...,0,10.1007/s43621-025-02349-5,https://www.scopus.com/inward/record.uri?eid=2...,"Faculty of Management and Economics, Salaam Un...","Mohamed, Mahad Abdiwali, Faculty of Management...",This paper conducts a bibliometric review of t...,Bibliometric study; Crowdfunding; Cryptocurren...,Springer Nature,Article,Scopus
1,"Ntuala, G.M.; Xia, Q.; Xia, H.; Badjie, A.; Mu...","Ntuala, Grace Mupoyi (60002650900); Xia, Qi (3...",60002650900; 35101036600; 36024522400; 6000246...,A dual-layer GNN with economic penalty mechani...,2026,Expert Systems with Applications,299,NaN,130121,NaN,...,0,10.1016/j.eswa.2025.130121,https://www.scopus.com/inward/record.uri?eid=2...,University of Electronic Science and Technolog...,"Ntuala, Grace Mupoyi, University of Electronic...",Blockchain fraud detection faces critical chal...,Blockchain fraud detection; Economic penalties...,Elsevier Ltd,Article,Scopus
2,"Cheah, J.E.-T.; Dao, T.; Do, H.; Mishra, T.","Cheah, Jeremy Eng Tuck (57219857925); Dao, Tho...",57219857925; 57372940900; 58606327800; 2293510...,Speed of Adjustment in Digital Assets in a Dec...,2026,Journal of Futures Markets,46,2,NaN,320.0,...,0,10.1002/fut.70055,https://www.scopus.com/inward/record.uri?eid=2...,"Nottingham Trent University, Nottingham, Notti...","Cheah, Jeremy Eng Tuck, Nottingham Trent Unive...",This paper investigates the stability and co-m...,blockchain technology; digital assets; ecentra...,John Wiley and Sons Inc,Article,Scopus
3,"Wang, G.; Hausken, K.","Wang, Guizhou (57221098821); Hausken, Kjell (3...",57221098821; 35240013300,Unravelling the global landscape of Bitcoin re...,2026,Technology Analysis and Strategic Management,38,1,NaN,1.0,...,5,10.1080/09537325.2024.2306931,https://www.scopus.com/inward/record.uri?eid=2...,"Faculty of Science and Technology, Universitet...","Wang, Guizhou, Faculty of Science and Technolo...",Bitcoin has been gaining increasing attention ...,bibliometric analysis; Bitcoin; blockchain; cr...,Routledge,Article,Scopus
4,"Kranz, T.; Schaaf, V.; Guggenberger, T.; Strük...","Kranz, Tobias T. (36809923900); Schaaf, Vincen...",36809923900; 59960318700; 57218403033; 6507170631,From Bricks to Blocks: Designing a Framework f...,2026,IEEE Access,14,NaN,NaN,4089.0,...,0,10.1109/ACCESS.2025.3648172,https://www.scopus.com/inward/record.uri?eid=2...,Fraunhofer Institute for Applied Information T...,"Kranz, Tobias T., Fraunhofer Institute for App...",Decentralized Finance (DeFi) promises to lay g...,Blockchain; DeFi; DSR; real-world asset; token...,Institute of Electrical and Electronics Engine...,Article,Scopus


In [ ]:
abstracts = df['Abstract']
titles = df['Title']
year = df['Year']

In [ ]:
len(abstracts)

899

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from torch import cuda

model_id = 'meta-llama/Llama-2-7b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

print(device)

cuda:0


In [ ]:
from torch import bfloat16
import transformers

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

In [ ]:
# Llama 2 Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# Llama 2 Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMS

In [ ]:
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

Device set to use cuda:0


In [ ]:
prompt = "Could you explain to me how 4-bit quantization works as if I am 5?"
res = generator(prompt)
print(res[0]["generated_text"])

Could you explain to me how 4-bit quantization works as if I am 5?
 Unterscheidung von 4-Bit-Quantisierung und 8-Bit-Quantisierung.
How does 4-bit quantization work?
What is the difference between 4-bit and 8-bit quantization?


In [ ]:
system_prompt = """
<s>[INST] <<SYS>>
You are an expert academic researcher specializing in entrepreneurship and cryptocurrency research. Your task is to create concise, descriptive labels for research topics based on academic abstracts.
<</SYS>>
"""

example_prompt = """
I have a topic that contains the following academic abstracts:
- This study examines how blockchain-based initial coin offerings (ICOs) provide alternative fundraising mechanisms for technology startups, analyzing token sale structures and investor participation patterns across 250 ICO campaigns.
- We investigate the determinants of successful token offerings in the cryptocurrency market, focusing on whitepaper quality, team composition, and regulatory compliance as signals of venture legitimacy.
- Our research explores how entrepreneurs utilize tokenization to attract capital, examining the relationship between token economics design and fundraising outcomes in decentralized finance ventures.

The topic is described by the following keywords: 'ICO, token, funding, investor, capital, offering, blockchain, fundraising, venture'.

Based on the information about the topic above, please create a short label of this topic. Make sure to only return the label and nothing more.

[/INST] ICO Fundraising & Token Economics
"""

main_prompt = """
[INST]
I have a topic that contains the following academic abstracts:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short, descriptive label of this research topic. The label should:
- Be concise
- Capture the core research focus
- Use academic terminology appropriate for entrepreneurship and cryptocurrency research
- Be suitable for a literature review taxonomy

Make sure to only return the label and nothing more.
[/INST]
"""

In [ ]:
prompt = system_prompt + example_prompt + main_prompt

In [ ]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("BAAI/bge-small-en")
embeddings = embedding_model.encode(abstracts, show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

In [ ]:
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=20, min_samples=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [ ]:
# Pre-reduce embeddings for visualization purposes
reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.1, metric='cosine', random_state=42).fit_transform(embeddings)

In [ ]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration

# KeyBERT
keybert = KeyBERTInspired()

# MMR
mmr = MaximalMarginalRelevance(diversity=0.3)

# Text generation with Llama 2
llama2 = TextGeneration(generator, prompt=prompt, doc_length=50, tokenizer=tokenizer)

# All representation models
representation_model = {
    "KeyBERT": keybert,
    "Llama2": llama2,
    "MMR": mmr
}

In [ ]:
topic_model = BERTopic(

  # Sub-models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(abstracts, embeddings)

2026-01-27 21:37:03,973 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-01-27 21:37:06,916 - BERTopic - Dimensionality - Completed ✓
2026-01-27 21:37:06,917 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-27 21:37:06,971 - BERTopic - Cluster - Completed ✓
2026-01-27 21:37:06,978 - BERTopic - Representation - Fine-tuning topics using representation models.
100%|██████████| 12/12 [01:27<00:00,  7.32s/it]
2026-01-27 21:38:36,416 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model.get_topic(1, full=True)["KeyBERT"]

[('fintech', np.float32(0.8815639)),
 ('blockchain', np.float32(0.84149516)),
 ('banking', np.float32(0.82886565)),
 ('cryptocurrency', np.float32(0.8243059)),
 ('ethereum', np.float32(0.82297266)),
 ('decentralized', np.float32(0.8224341)),
 ('finance', np.float32(0.82186216)),
 ('research', np.float32(0.8178723)),
 ('technology', np.float32(0.814172)),
 ('financial', np.float32(0.81124127))]

In [ ]:
llama2_labels = [label[0][0].split("\n")[0] for label in topic_model.get_topics(full=True)["Llama2"].values()]
topic_model.set_topic_labels(llama2_labels)

In [ ]:
#topic_model.visualize_documents(titles, reduced_embeddings=reduced_embeddings, hide_annotations=True, hide_document_hover=False, custom_labels=True)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,CustomName,Representation,KeyBERT,Llama2,MMR,Representative_Docs
0,-1,137,-1_the_and_of_to,Blockchain-Based Digital Finance,"[the, and, of, to, in, for, blockchain, on, is...","[blockchain, cryptocurrencies, cryptocurrency,...","[Blockchain-Based Digital Finance, , , , , , ,...","[blockchain, cryptocurrency, cryptocurrencies,...",[Blockchain for business is a new concept whic...
1,0,289,0_the_and_blockchain_of,Blockchain and Cryptocurrency Research Topic L...,"[the, and, blockchain, of, to, in, technology,...","[blockchain, cryptocurrencies, decentralized, ...",[Blockchain and Cryptocurrency Research Topic ...,"[blockchain, to, technology, it, data, securit...",[Blockchain technology (BCT) has perhaps gaine...
2,1,83,1_the_and_of_fintech,Fintech Innovation and Decentralization: A Stu...,"[the, and, of, fintech, to, in, financial, thi...","[fintech, blockchain, banking, cryptocurrency,...",[Fintech Innovation and Decentralization: A St...,"[the, fintech, ai, token, blockchain, technolo...","[Today, the financial technology market (finte..."
3,2,83,2_defi_the_and_to,Decentralized Finance (DeFi): A Literature Review,"[defi, the, and, to, of, in, decentralized, fi...","[decentralised, decentralized, blockchain, def...",[Decentralized Finance (DeFi): A Literature Re...,"[defi, decentralized, finance, liquidity, cryp...",[The Bitcoin whitepaper [1] published in 2008 ...
4,3,60,3_ico_icos_the_of,ICO Fundraising & Tokenomics,"[ico, icos, the, of, and, initial, in, to, coi...","[ico, icos, cryptocurrency, crowdfunding, inve...","[ICO Fundraising & Tokenomics, , , , , , , , , ]","[ico, icos, in, coin, success, investors, toke...",[The article deals with a new financial tool o...
5,4,49,4_the_and_of_price,Cryptocurrency Market Volatility Prediction an...,"[the, and, of, price, to, cryptocurrency, bitc...","[cryptocurrencies, cryptocurrency, bitcoin, bl...",[Cryptocurrency Market Volatility Prediction a...,"[cryptocurrency, bitcoin, trading, prediction,...",[Featured Application: The confidence-threshol...
6,5,48,5_the_to_of_and,Cryptocurrency Fraud Detection using Blockchai...,"[the, to, of, and, in, transactions, on, we, f...","[blockchain, cryptocurrency, ethereum, bitcoin...",[Cryptocurrency Fraud Detection using Blockcha...,"[transactions, scam, blockchain, graph, detect...",[The prosperity of the cryptocurrency ecosyste...
7,6,47,6_the_of_and_to,"Certainly! Based on the provided information, ...","[the, of, and, to, in, regulatory, legal, for,...","[cryptocurrency, cryptocurrencies, blockchain,...","[Certainly! Based on the provided information,...","[of, regulatory, legal, cryptocurrency, regula...",[The chapter highlights some aspects of the pr...
8,7,29,7_crowdfunding_the_to_and,Crowdfunding and Blockchain Technologies for E...,"[crowdfunding, the, to, and, of, is, for, bloc...","[crowdfunding, blockchain, cryptocurrency, eth...",[Crowdfunding and Blockchain Technologies for ...,"[crowdfunding, blockchain, people, technology,...",[The crowd sourcing platform is a website that...
9,8,28,8_the_adoption_and_of,Cryptocurrency Adoption and Acceptance,"[the, adoption, and, of, cryptocurrency, facto...","[cryptocurrency, cryptocurrencies, blockchain,...",[Cryptocurrency Adoption and Acceptance\n\nLab...,"[cryptocurrency, factors, model, cryptocurrenc...",[Purpose: Financial inclusion provides access ...


In [ ]:
#topic_info = topic_model.get_topic_info()
#topic_info.to_csv('bertopic_topics.csv', index=False)
#print('Topic information saved to bertopic_topics.csv')

In [ ]:
df['Topic'] = topics
df['Probability'] = probs

In [ ]:
# 1. Built-in temporal visualization
years = df['Year'].tolist()
topics_over_time = topic_model.topics_over_time(abstracts, years)
fig = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=14)
fig.write_html('all_topics_over_time.html')

12it [00:00, 15.17it/s]


In [ ]:
# 2. Simple period comparison
df['Period'] = df['Year'].apply(lambda y:
    "Early (2017-2019)" if y <= 2019 else
    "Mid (2020-2022)" if y <= 2022 else
    "Recent (2023+)")

In [ ]:
period_topic_table = df.groupby(['Period', 'Topic']).size().unstack(fill_value=0)
print("\n=== Papers per Topic per Period ===")
print(period_topic_table)


=== Papers per Topic per Period ===
Topic              -1    0    1    2    3    4    5    6    7    8    9    10
Period                                                                       
Early (2017-2019)   26   54   10    0   10    3    1    9    4    2    6    6
Mid (2020-2022)     26   97   22   15   34   14   12   10    5   11    7    3
Recent (2023+)      85  138   51   68   16   32   35   28   20   15   12   12


In [ ]:
topic_labels_dict = {i: topic_model.custom_labels_[i] for i in range(len(topic_model.custom_labels_))}
period_topic_table.columns = [topic_labels_dict.get(col, f"Topic {col}") for col in period_topic_table.columns]
print("\n=== Papers per Topic per Period (with labels) ===")
print(period_topic_table)


=== Papers per Topic per Period (with labels) ===
                   Topic -1  Blockchain-Based Digital Finance  \
Period                                                          
Early (2017-2019)        26                                54   
Mid (2020-2022)          26                                97   
Recent (2023+)           85                               138   

                   Blockchain and Cryptocurrency Research Topic Label:  \
Period                                                                   
Early (2017-2019)                                                 10     
Mid (2020-2022)                                                   22     
Recent (2023+)                                                    51     

                   Fintech Innovation and Decentralization: A Study of Blockchain and AI's Impact on Financial Services  \
Period                                                                                                                    
Early 

In [ ]:
period_topic_pct = df.groupby(['Period', 'Topic']).size().groupby(level=0).apply(lambda x: 100 * x / x.sum()).unstack(fill_value=0)
period_topic_pct.columns = [topic_labels_dict.get(col, f"Topic {col}") for col in period_topic_pct.columns]
print("\n=== Percentage of Papers per Period ===")
print(period_topic_pct.round(1))


=== Percentage of Papers per Period ===
                                     Topic -1  \
Period            Period                        
Early (2017-2019) Early (2017-2019)      19.8   
Mid (2020-2022)   Mid (2020-2022)        10.2   
Recent (2023+)    Recent (2023+)         16.6   

                                     Blockchain-Based Digital Finance  \
Period            Period                                                
Early (2017-2019) Early (2017-2019)                              41.2   
Mid (2020-2022)   Mid (2020-2022)                                37.9   
Recent (2023+)    Recent (2023+)                                 27.0   

                                     Blockchain and Cryptocurrency Research Topic Label:  \
Period            Period                                                                   
Early (2017-2019) Early (2017-2019)                                                7.6     
Mid (2020-2022)   Mid (2020-2022)                                      

In [ ]:
topic_summary = []
for topic in df['Topic'].unique():
    if topic == -1:  # Skip outliers
        continue

    topic_papers = df[df['Topic'] == topic]

    summary = {
        'Topic_Number': topic,
        'Topic_Label': topic_labels_dict[topic],
        'Total_Papers': len(topic_papers),
        'First_Year': topic_papers['Year'].min(),
        'Last_Year': topic_papers['Year'].max(),
        'Peak_Year': topic_papers.groupby('Year').size().idxmax(),
        'Peak_Count': topic_papers.groupby('Year').size().max(),
        'Early_Period_Papers': len(topic_papers[topic_papers['Period'] == 'Early (2017-2019)']),
        'Mid_Period_Papers': len(topic_papers[topic_papers['Period'] == 'Mid (2020-2021)']),
        'Recent_Period_Papers': len(topic_papers[topic_papers['Period'] == 'Recent (2022+)']),
        'Recent_Percentage': (len(topic_papers[topic_papers['Period'] == 'Recent (2022+)']) / len(topic_papers) * 100),
        'Average_Citations': topic_papers['Cited by'].mean() if 'Cited by' in topic_papers.columns else None
    }

    topic_summary.append(summary)

topic_summary_df = pd.DataFrame(topic_summary)
topic_summary_df = topic_summary_df.sort_values('Total_Papers', ascending=False)
topic_summary_df.to_csv('topic_summary_statistics.csv', index=False)
print("✓ Saved: topic_summary_statistics.csv")

✓ Saved: topic_summary_statistics.csv
